#Setup

##Imports

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import sys
sys.path.append('/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch')

import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import json
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
from scipy import spatial
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
import torchvision.transforms.functional as F
from matplotlib import cm as CM
import random
from torchvision import datasets, transforms
transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],          std=[0.229, 0.224, 0.225]),
                   ])
                          
%matplotlib inline

Mounted at /content/gdrive


##Paths

In [ ]:
root = '/content/gdrive/MyDrive/TFE_crowd_counting'
dataset_path = os.path.join(root,'Dataset')
images_path = os.path.join(dataset_path,'images')
path_sets = [images_path]  #All images at the same place
scripts_path = os.path.join(root,'CSRNet-pytorch')

img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

#Utils
This section contains : 

1.   a method to load a model
2.   a method to get a estimation of the number of people and the density map based on an image



In [ ]:
#return the loaded model located at model_path (or the basic shangai partAmodel if no path is given ) 
#using gpu or using cpu (if the use_gpu parameter is set to False)
def load_model(model_path = '/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/model_best.pth.tar', use_gpu = False):
  from torchvision import datasets, transforms
  transform=transforms.Compose([
                        transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225]),
                    ])
  model = CSRNet()
  if use_gpu:
    model = model.cuda()
  else:
    model = model.cpu()
  checkpoint = torch.load(model_path)
  model.load_state_dict(checkpoint['state_dict'])
  return model

In [ ]:
model = load_model()

#To plot the image density map from the output, use : 
#plt.imshow(np.squeeze(output.detach().cpu().numpy(),(0,1)),cmap=CM.jet)
def prediction(model,image_path, use_gpu = True):
  img = transform(Image.open(image_path).convert('RGB')).cuda()
  output = model(img.unsqueeze(0))
  people_nbr = int(output.detach().cpu().sum().numpy())
  return people_nbr, output


<class 'dict'>


In [ ]:
print(model)

CSRNet(
  (frontend): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

#Evaluation and Visualization

In [ ]:
#Load models
model1 = load_model('/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/PartAmodel_best.pth.tar') #model shangai part1
model2 = load_model(model_path ='/content/gdrive/MyDrive/TFE_crowd_counting/CSRNet-pytorch/model_best.pth.tar') #model trained on barb94

In [ ]:
#calculate and print the MAE of the models
def MAE_eval(model):
  mae = 0
  for i in range(len(img_paths)):
      img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
      gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground_truth'),'r')
      groundtruth = np.asarray(gt_file['density'])
      output = model(img.unsqueeze(0))
      mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
      
  return mae/len(img_paths)

print(MAE_eval(model))
print(MAE_eval(model2))

4.313128113746642
1.5741261596029452


In [ ]:
#print estimated number and real number and show's the images and density representations
indexes = [5,18,19,22,36]
plt.figure(figsize = (100,100))
for i in range(len(indexes)):
    gt_file = h5py.File(img_paths[indexes[i]].replace('.jpg','.h5').replace('images','ground_truth'),'r')
    groundtruth = np.asarray(gt_file['density'])
    nbr1, output = prediction(model,img_paths[indexes[i]])
    nbr2, output2= prediction(model2,img_paths[indexes[i]])
    
    plt.subplot(5,4,i*4+1)
    plt.imshow(Image.open(img_paths[indexes[i]]))
    plt.title("Raw image", fontsize=75)

    plt.subplot(5,4,i*4+2)
    plt.imshow(groundtruth,cmap=CM.jet)
    plt.title("Groundtruth : " + str(int(np.sum(groundtruth))), fontsize=75)

    plt.subplot(5,4,i*4+3)
    plt.imshow(np.squeeze(output.detach().cpu().numpy(),(0,1)),cmap=CM.jet)
    plt.title("Model prediction : " + str(nbr1), fontsize=75)

    
    plt.subplot(5,4,i*4+4)
    plt.imshow(np.squeeze(output2.detach().cpu().numpy(),(0,1)),cmap=CM.jet)
    plt.title("Model prediction : " + str(nbr2), fontsize=75)

plt.show()

Output hidden; open in https://colab.research.google.com to view.